In [5]:
from IPython.display import display_javascript

def make_cell(s):

    display_javascript("""var t_cell = IPython.notebook.insert_cell_below()
    t_cell.set_text('<!--\\n' + t_cell.get_text() + '\\n--> \\n{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();""".format(s.replace('\n','\\n')), raw=True)

# Introduction


So, you have somehow found micropython, fallen in love with it in an instance, broken your piggy bank, and run off, head over heels, to buy a pyboard. You have probably paid extra for the expedited shipping. Once the pyboard arrived, you became excited like a puppy with a bone. You played with the hardware, learnt how to access the accelerometer, switch, LEDs, and temperature sensor, and you have successfully communicated with other devices via the I2C, SPI, USART, or CAN interfaces. You have plugged the board in a computer, and driven someone crazy by emulating a seemingly disoriented mouse on it. You have written your own python functions, even compiled them into frozen modules, and burnt the whole damn thing onto the microcontroller. Then you have toyed with the on-board assembler, because you hoped that you could gain some astronomical factors in speed. (But you couldn't.)

And yet, after all this, you feel somewhat unsatisfied. You find that you want to access the periphery in a special way, or you need some fancy function that, when implemented in python itself, seems to consume too much RAM, and takes an eternity to execute, and assembly, especially with its limitations, is just far too awkward. Or perhaps, you are simply dead against making your code easily readable by writing everything in python, and you want to hide the magic, just for the heck of it. But you still want to retain the elegance of python. 

If, after thorough introspection and soul-searching, you have discovered these latter symptoms in yourself, you have two choices: either you despair, scrap your idea, and move on, or you learn how the heavy lifting behind the python facade is done, and spin your own functions, classes, and methods in C. As it turns out, it is not that hard, once you get the hang of it. The sole trick is to get the hang of it. The fact that the micropython code base is organised in a rather logical fashion certainly makes matters feasible. 

On the following pages I would like to show how new functionality can be added and exposed to the python interpreter. I will try to discuss all aspects of micropython in an approachable way. Each concept will be presented in a minimal implementation that you can compile right away, and try yourself. At the end of each chapter, I will list the discussed code in its entirety, so that copying and pasting does not involve copious amounts of work. The code, as well as the source of the documentation are also available under https://github.com/v923z/micropython-development.

I start out with a very simple module and slowly build upon it. At the end of the discussion, I will outline my version of a general-purpose math library, similar to numpy. In fact, when working on this math module, I realised that a decent programming guide to micropython is sorely missing, hence this document. Obviously, numpy is a huge library, and we are not going to implement all aspects of it. But we will be able to define efficiently stored arrays on which we can do vectorised computations, work with matrices, invert and contract them, fit polynomials to measurement data, and get the Fourier transform of an arbitrary sequence. I hope you will enjoy the ride!

One last comment: I believe, all examples in this document could be implemented with little effort in python itself, and I am definitely not advocating the inclusion of such trivial cases in the firmware. I chose these examples on two grounds: First, they are all simple, almost primitive, but for this very reason, they demonstrate an idea without distraction. Second, having a piece of parallel python code is useful insofar as it tells us what to expect, and we can compare the results of the C implemetation to that of the native firmware.

# Setting up the environment, compiling the code

## General remarks

I have written this document entirely as an jupyter notebook. There are several reasons for this. First, the prompt visual feedback on markdown text, and python/C code. Second, for demonstration purposes, I did need to run python code either on the local machine, or the pyboard, and it was simply natural to do that via jupyter. Third, and perhaps most importantly, by using a notebook, I could work from a single location: the documentation, the C code and its compilation, and the tests (either on the unix port, or on the pyboard) are all in a single container. It is impossible to overstate the advantages of this. 

## User modules in micropython

Beginning with the 1.10 version of micropython, it became quite simple to add a user-defined C module to the firmware. You simply drop two or three files in an arbitrary directory, and pass a compiler flag to `make`. 

-------------------------------

```bash

make USER_C_MODULES=../../../user_modules CFLAGS_EXTRA=-DMODULE_EXAMPLE_ENABLED=1 all
```
or 

```bash

make USER_C_MODULES=../../../user_modules all
```
When developing user modules, it definitely makes sense to have a directory separate from micropython itself, because it is much easier to keep track of changes.

The micropython codebase itself is set up a rather modular way. Provided you cloned the micropython repository with 

```bash
git clone https://github.com/micropython/micropython.git 
```
onto your  you look at the top level directories, you will see something like this:

In [5]:
!ls ../../micropython/

ACKNOWLEDGEMENTS    docs      extmod   logo	  py	     tools
CODECONVENTIONS.md  drivers   lib      mpy-cross  README.md
CONTRIBUTING.md     examples  LICENSE  ports	  tests


Out of all the directoties, at least two are of particular interest. Namely, `/py/`, where the python interpreter is implemented, and `/ports/`, which contains the hardware-specific files. All questions pertaining to programming micropython in C can be answered by browsing these two directories, and perusing the relevant files therein. 

# micropython internals

Before exploring the exciting problem of micropython function implementation in C, we should first understand how python objects are stored and treated at the firmware level. 

## Object representation

Whenever you write 

```python
>>> a = 1
>>> b = 1
>>> a + b
```

on the python console, first the two new variables, `a`, and `b` are created and a reference to them is stored in memory. Then the value of 1 will be associated with these variables. In the last line, when the sum is to be computed, the interpreter somehow has to figure out, how to decipher the values stored in `a`, and `b`: in the RAM, these two variables are just bytes, but depending on the type of the variable, different meanings will be associated with these bytes. Since the type cannot be known at compile time, there must be a mechanism for keeping stock of this extra piece of information. This is, where `mp_obj_t`, defined in `obj.h`, takes centre stage. 

If you look at any of the C functions that are exposed to the python interpreter, you will see something like this

```c
mp_obj_t some_function(mp_obj_t some_variable, ...) {
    // some_variable is converted to fundamental C types (bytes, ints, floats, pointers, structures, etc.)
    ...
}
```
Variables of type `mp_obj_t` are passed to the function, and the function returns the results as an object of type `mp_obj_t`. So, what is all this fuss this about? Basically, `mp_obj_t` is nothing but an 8-byte segment of the memory, where all concrete objects are encoded. There can be various object encodings. E.g., in the `A` encoding, integers are those objects, whose rightmost bit in this 8-byte representation is set to 1, and the value of the integer can then be retrieved by shifting these 8 bytes by one to the right. In the `B` encoding, the variable is an integer, if its value is 1, when ANDed with 3, and the value will be returned, if we shift the 8 bytes by two to the right. 

## Type checking

Fortunately, we do not have to be concerned with the representations and the shifts, because there are pre-defined macros for such operations. So, if we want to find out, whether `some_variable` is an integer, we can inspect the value of the Boolean 

```c
MP_OBJ_IS_SMALL_INT(some_variable)
```

The integer value stored in `some_variable` can then be gotten by calling `MP_OBJ_SMALL_INT_VALUE`: 

```c
int value_of_some_variable = MP_OBJ_SMALL_INT_VALUE(some_variable);
```

These decoding steps take place somewhere in the body of `some_function`, before we start working with native C types. Once we are done with the calculations, we have to return an `mp_obj_t`, so that the interpreter can handle the results (e.g., show it on the console, or pipe it to the next instruction). In this case, the encoding is done by calling 

```c
mp_obj_new_int(value_of_some_variable)

```

More generic types can be treated with the macro `MP_OBJ_IS_TYPE`, which takes the object as the first, and a pointer to the type as the second argument. So, if you want to find out, whether `some_variable` is a tuple, you could apply the `MP_OBJ_IS_TYPE` macro, 

```c
MP_OBJ_IS_TYPE(myobject, &mp_type_tuple)
```

While the available types can be found in `obj.h`, they all follow the `mp_type_` + python type pattern, so in most cases, it is not even necessary to look them up. We should also note that, when defined properly, `MP_OBJ_IS_TYPE` can be called on objects, that are defined in the user module itself, i.e., you can have your own data type, and 

```c
MP_OBJ_IS_TYPE(myobject, &my_type)
```

will just work. We return to this question later. 

## python constants

At this point, we should mention python constants,`True`, `False`, `None` and the like are also defined in `obj.h`. These are objects of type `mp_obj_t`, as almost anything else, so you can return them from a function, when the function is meant to return directly to the interpreter.

# Developing your first module

Having seen, what the python objects look like to the interpreter, we can start with out explorations in earnest. We begin by adding a simple module to the python interpreter. The module will have a single function that takes two numbers, and adds them. I know that this is the most exciting thing since sliced bread, and you always wondered, why there isn't a built-in python function for such an exciting task. Well, wonder no more! From now on, *your* micropython will have one. 

First I show the file in its entirety (20 something lines all in all), and then discuss the parts.

```c
#include "py/obj.h"
#include "py/runtime.h"

STATIC mp_obj_t simplefunction_add_ints(mp_obj_t a_obj, mp_obj_t b_obj) {
    int a = mp_obj_get_int(a_obj);
    int b = mp_obj_get_int(b_obj);
    return mp_obj_new_int(a + b);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);

STATIC const mp_rom_map_elem_t simplefunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_simplefunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), MP_ROM_PTR(&simplefunction_add_ints_obj) },
};
STATIC MP_DEFINE_CONST_DICT(simplefunction_module_globals, simplefunction_module_globals_table);

const mp_obj_module_t simplefunction_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&simplefunction_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_simplefunction, simplefunction_user_cmodule, MODULE_SIMPLEFUNCTION_ENABLED);

```

You should be able to compile the module above by calling 

```bash
make USER_C_MODULES=../../../usermod/ all
```

## Header files

A module will not be too useful without at least two includes: `py/obj.h`, where all the relevant constants and macros are defined, and `py/runtime.h`, which contains the declaration of the interpreter. 

## Defining user functions

After including the necessary headers, we define the function that is going to do the heavy lifting. By passing variables of `mp_obj_t` type, we make sure that the function will be able to accept values from the python console. If you happen to have an internal helper function in your module that is not exposed in python, you can pass whatever type you need. Similarly, by returning an object of `mp_obj_t` type, we make the results visible to the interpreter, i.e., we can assign the value returned to variables. 

The downside of passing `mp_obj_t`s around is that you cannot simply assign them to usual C variables, i.e., when you want to operate on them, you have to extract the values first. This is why we have to invoke the `mp_obj_get_int()` function, and conversely, before returning the results, we have to do a type conversion to `mp_obj_t` by calling `mp_obj_new_int()`. These are the decoding/encoding steps that we discussed above.

### Referring to user functions

We have now a function that should be sort of OK, but the python interpreter still cannot work with. For that, we have to turn our function into a function object. This is what happens in the line

```c
STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);
```

The first argument is the name of the function object to which our actual function, the last argument, will be bound. Now, these `MP_DEFINE_CONST_FUN_OBJ_*` macros, defined in the header file `py/obj.h` (one more reason not to forget about `py/obj.h`), come in seven flavours, depending on what kind of, and how many arguments the function is supposed to take. In the example above, our function is meant to take two arguments, hence the 2 at the end of the macro name. Functions with 0 to 4 arguments can be bound in this way.

But what, if you want a function with more than four arguments, as is the case many a time in python? Under such circumstances, one can make use of the 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR(obj_name, n_args_min, fun_name)
```

macro, where the second argument, an integer, gives the minimum number of arguments. The number of arguments can be bound from above by wrapping the function with 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(obj_name, n_args_min, n_args_max, fun_name) 
```

Later we will see, how we can define functions that can also take keyword arguments.

At this point, we are more or less done with the C implementation of our function, but we still have to expose it. This we do by adding a table, an array of key/value pairs to the globals of our module, and bind the table to the `_module_globals` variable by applying the `MP_DEFINE_CONST_DICT` macro. This table should have at least one entry, the name of the module, which is going to be stored in the string `MP_QSTR___name__`. 

The other keys of the table are the pointers to the functions that we implemented, and the names that we want to call these functions in python itself. So, in the example below, our `simplefunction_add_ints` function will be invoked, when we call `add_ints` in the console.


```c
STATIC const mp_rom_map_elem_t simplefunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_simplefunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), MP_ROM_PTR(&simplefunction_add_ints_obj) },
};
STATIC MP_DEFINE_CONST_DICT(simplefunction_module_globals, simplefunction_module_globals_table);
```
This three-step pattern is common to all function implementations, so I repeat it here: 

1. implement the function
2. then turn it into a function object
3. and finally, register the function in the name space of the module

It doesn't matter, whether our function takes positional arguments, or keyword argument, or both, these are the required steps.

Having defined the function object, we have finally to register the function with 

```c
MP_REGISTER_MODULE(MP_QSTR_simplefunction, simplefunction_user_cmodule, MODULE_SIMPLEFUNCTION_ENABLED);
```

This last line is particularly useful, because by setting the `MODULE_SIMPLEFUNCTION_ENABLED` variable in `mpconfigport.h`, you can selectively exclude modules from the compilation, i.e., if in `mpconfigport.h`, which should be in the root directory of the port you want to compile for, 

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (1)
```
then `simplefunction` will be included in the firmware, while with

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (0)
```

the module will be dropped, even though the source is in your modules folder.

## Function names

Whenever you want your function to be visible from the interpreter, you have to add the function object to a dictionary. The dictionary 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_somefunction), MP_ROM_PTR(&test_somefunction_obj) },
};


STATIC MP_DEFINE_CONST_DICT(test_locals_dict, test_locals_dict_table);
```

# Error handling



## Parsing arguments

## Parsing keyword arguments

One of the most useful features of python is that functions can accept positional as well as keyword arguments, thereby providing a very flexible interface for feeding values to a function. In this section, we will see how we should set up our function, so that it can interpret what we passed in. 

It does not matter, whether we have positional or keyword arguments, at one point, the interpreter has to turn all arguments into a deterministic sequence of objects. We stipulate this sequence in the constant variable called `allowed_args[]`. This is an array of type `mp_arg_t`, which is nothing but a structure with two `uint16` values, and a union named `mp_arg_val_t`. This union holds the default value and the type of the variable that we want to pass. The `mp_arg_t` structure, defined in `runtime.h`, looks like this:

```c
typedef struct _mp_arg_t {
    uint16_t qst;
    uint16_t flags;
    mp_arg_val_t defval;
} mp_arg_t;
```

The last member, `mp_arg_val_t` is 

```c
typedef union _mp_arg_val_t {
    bool u_bool;
    mp_int_t u_int;
    mp_obj_t u_obj;
    mp_rom_obj_t u_rom_obj;
} mp_arg_val_t;
```

### Keyword arguments with numerical values

In our first example `keywordfunction.c`, we define a summing function with two arguments. 

```c
#include "py/obj.h"
#include "py/runtime.h"
#include "py/builtin.h"

STATIC mp_obj_t keywordfunction_add_ints(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {

	static const mp_arg_t allowed_args[] = {
		{ MP_QSTR_a, MP_ARG_INT | MP_ARG_REQUIRED, {.u_int = 0} }, 
        { MP_QSTR_b, MP_ARG_KW_ONLY | MP_ARG_INT | MP_ARG_REQUIRED, {.u_int = 0} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
	int16_t a = args[0].u_int;
	int16_t b = args[1].u_int;
	printf("a = %d, b = %d\n", a, b);
	return mp_obj_new_int(a + b);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_KW(keywordfunction_add_ints_obj, 2, keywordfunction_add_ints);

STATIC const mp_rom_map_elem_t keywordfunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_keywordfunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), (mp_obj_t)&keywordfunction_add_ints_obj },
};

STATIC MP_DEFINE_CONST_DICT(keywordfunction_module_globals, keywordfunction_module_globals_table);

const mp_obj_module_t keywordfunction_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&keywordfunction_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_keywordfunction, keywordfunction_user_cmodule, MODULE_KEYWORDFUNCTION_ENABLED);
```

One convenience of a function with keyword arguments is that we do not have to care about the arguments in the C implementation: the argument list is always the same, and it is passed in by the interpreter: the number of arguments of the python function, an array with the positional arguments, and a map for the keyword arguments. 

After parsing the arguments with `mp_arg_parse_all`, whatever was at the zeroth position of `allowed_args[]` will be called `args[0]`, the object at the first position of `allowed_args[]` will be turned into `args[1]`, and so on. 

This is, where we also define, what the name of the keyword argument is going to be: whatever comes after `MP_QSTR_`. But hey, presto! The name should be an integer with 16 bits, shouldn't it? After all, this is the first member of `mp_arg_t`. So what is going on here? Well, for the efficient use of RAM, all MP_QSTRs are turned into `unint16_t` internally. This applies not only to the names in functions with keyword arguments, but also for module and function names, in the `_module_globals_table[]`.

The second member of the `mp_arg_t` structure is the flags that determine, e.g., whether the argument is required, if it is of integer or `mp_obj_t` type, and whether it is positional or a keyword argument. These flags can be combined by ORing them, as we done in the example above. 

The last member in `mp_arg_t` is the default value. Since this is a member variable, when we make use of it, we have to extract the value by adding `.u_int` to the argument. 

When turning our function into a function object, we have to call a special macro, `MP_DEFINE_CONST_FUN_OBJ_KW`, defined in `obj.h`, which is somewhat similar to `MP_DEFINE_CONST_FUN_OBJ_VAR`: in addition to the function object and the function, one also has to specify the minimum number of arguments in the python function.

Other examples on passing keyword arguments can be found in some of the hardware implementation files, e.g., `ports/stm32/pyb_i2c.c`, or `ports/stm32/pyb_spi.c`.

### Keyword arguments with arbitrary values

We have seen how integer values can be extracted from keyword arguments, but unfortunately, that method is going to get you only that far. What if we want to pass something more complicated, in particular a string, or a list, or some other non-trivial python type? This is what we are going to look at now. 

# Working with classes

Of course, python would not be python without classes. A module can also include the implementation of classes. The procedure is similar to what we have already seen in the context of standard functions, except that we have to define a structure that holds at least a string with the name of the class, a pointer to the initialisation and printout functions, and a local dictionary. A typical structure would look like

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_method1), MP_ROM_PTR(&test_method1_obj) },
	{ MP_ROM_QSTR(MP_QSTR_method2), MP_ROM_PTR(&test_method2_obj) },
    ...                                                           
}

const mp_obj_type_t simpleclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_simpleclass,
	.print = simpleclass_print,
	.make_new = simpleclass_make_new,
	.locals_dict = (mp_obj_dict_t*)&simpleclass_locals_dict,
};
```

The locals dictionary, `.locals_dict`, contains all user-facing methods and constants of the class, while the `myclass_type` structure's `name` member is what our class is going to be called. `.print` is roughly the equivalent of `__str__`, `.make_new` is the C name for `__init__`. 

In order to see how this all works, we are going to implement a very simple class, which holds two integer variables, and has a method that returns the sum of these two variables. In python, a possible realisation could look like this:

In [10]:
class myclass:
    
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def mysum(self):
        return self.a + self.b
    
    
A = myclass(1, 2)
A.mysum()

3

In addition to the class implementation above and to show how class methods and regular functions can live in the same module, we will also have a function, which is not bound to the class itself, and which adds the two components in the class, i.e., that is similar to 

In [11]:
def add(class_instance):
    return class_instance.a + class_instance.b

add(A)

3

(Note that retrieving values from the class in this way is not exactly elegant, nor is it pythonic. We would usually implement a getter method for that.)

```c
#include "py/runtime.h"
#include "py/obj.h"

typedef struct _simpleclass_myclass_obj_t {
	mp_obj_base_t base;
	int16_t a;
	int16_t b;
} simpleclass_myclass_obj_t;

const mp_obj_type_t simpleclass_myclass_type;

STATIC void myclass_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
	(void)kind;
	simpleclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	mp_print_str(print, "myclass(");
	printf("%d, ", self->a);
	printf("%d)", self->b);
}

STATIC mp_obj_t myclass_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
	mp_arg_check_num(n_args, n_kw, 2, 2, true);
	simpleclass_myclass_obj_t *self = m_new_obj(simpleclass_myclass_obj_t);
	self->base.type = &simpleclass_myclass_type;
	self->a = mp_obj_get_int(args[0]);
	self->b = mp_obj_get_int(args[1]);
	return MP_OBJ_FROM_PTR(self);
}

// Class methods
STATIC mp_obj_t myclass_sum(mp_obj_t self_in) {
	simpleclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	return mp_obj_new_int(self->a + self->b);
}

MP_DEFINE_CONST_FUN_OBJ_1(myclass_sum_obj, myclass_sum);

STATIC const mp_rom_map_elem_t myclass_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_mysum), MP_ROM_PTR(&myclass_sum_obj) },
};

STATIC MP_DEFINE_CONST_DICT(myclass_locals_dict, myclass_locals_dict_table);

const mp_obj_type_t simpleclass_myclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_simpleclass,
	.print = myclass_print,
	.make_new = myclass_make_new,
	.locals_dict = (mp_obj_dict_t*)&myclass_locals_dict,
};

// Module functions
STATIC mp_obj_t simpleclass_add(const mp_obj_t o_in) {
	simpleclass_myclass_obj_t *class_instance = MP_OBJ_TO_PTR(o_in);
	return mp_obj_new_int(class_instance->a + class_instance->b);
}

MP_DEFINE_CONST_FUN_OBJ_1(simpleclass_add_obj, simpleclass_add);

STATIC const mp_map_elem_t simpleclass_globals_table[] = {
	{ MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_simpleclass) },
	{ MP_OBJ_NEW_QSTR(MP_QSTR_myclass), (mp_obj_t)&simpleclass_myclass_type },	
	{ MP_OBJ_NEW_QSTR(MP_QSTR_add), (mp_obj_t)&simpleclass_add_obj },
};

STATIC MP_DEFINE_CONST_DICT (
	mp_module_simpleclass_globals,
	simpleclass_globals_table
);

const mp_obj_module_t simpleclass_user_cmodule = {
	.base = { &mp_type_module },
	.globals = (mp_obj_dict_t*)&mp_module_simpleclass_globals,
};

MP_REGISTER_MODULE(MP_QSTR_simpleclass, simpleclass_user_cmodule, MODULE_SIMPLECLASS_ENABLED);
```

In `my_print`, we used the C function `printf`, but better options are also available. `mpprint.c` has a number of methods for printing all kinds of python objects. 

One more thing to note: the functions that are pointed to in `simpleclass_myclass_type` are not registered with the macro `MP_DEFINE_CONST_FUN_OBJ_VAR` or similar. The reason for this is that this automatically happens: `myclass_print` does not require user-supplied arguments beyond `self`, so it is known what the signature should look like. In `myclass_make_new`, we inspect the argument list, when calling 
```c
mp_arg_check_num(n_args, n_kw, 2, 2, true);
```
so, again, there is no need to turn our function into a function object. 

## Class special methods

Python has a number of special methods, which will make a class behave as a native object. So, e.g., if a class implements the `__add__(self, other)` method, then instances of that class can be added with the `+` operator. Here is an example in python:

In [4]:
class Adder:
    
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        self.value = self.value + other.value
        return self

a = Adder(1)
b = Adder(2)

c = a + b
c.value

3

Note that, while the above example is not particularly useful, it proves the point: upon calling the `+` operator, the values of `a`, and `b` are added. If we had left out the implementation of the `__add__` method, the python interpreter would not have a clue as to what to do with the objects. You can see for yourself, how sloppiness makes itself manifest:

In [5]:
class Adder:
    
    def __init__(self, value):
        self.value = value

a = Adder(1)
b = Adder(2)

c = a + b
c.value

TypeError: unsupported operand type(s) for +: 'Adder' and 'Adder'

Indeed, we do not support the `+` operator.

Now, the problem is that in the C implementation, these special methods have to be treated in a special way. The naive approach would be to add the pointer to the function to the locals dictionary as 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR___add__), MP_ROM_PTR(&test_add_obj) },
};
```
but that would not work. Well, this is not entirely true: the `+` operator would not work, but one could still call the method explicitly as

```python
a = Adder(1)
b = Adder(2)

a.__add__(b)

```

Before we actually add the `+` operator to our class, we should note that there are two kinds of special methods, namely the unary and the binary operators. 

In the first group are those, whose sole argument is the class instance itself. Two frequently used cases are the length operator, `len`, and `bool`. So, e.g., if your class implements the `__len__(self)` method, and the method returns an integer, then you can call the `len` function in the console

```python
len(myclass)
```

In the second category of operators are those, which require a left, as well as a right hand side: the left hand side is the class instance itself, while the right hand side can, in principle, be another instance of the same class, or some other type. An example for this was the `__add__` method in our `Adder` class. To prove that the right hand side needn't be of the same type, think of the *multiplication* of lists: 

```python
[1, 2, 3]*5
```

is perfectly valid, and has a well-defined meaning. The complete list of unary, as well as binary operators can be found in `runtime.h`. 

The module below implements five special methods altogether. Two unary, namely, `bool`, and `len`, and three binary operators, `==`, `+`, and `*`. Since the addition and multiplication will return a new instance of `myclass`, we define a new function, `create_new_class`, that, well, creates a new instance of `myclass`, and initialises the members with the two input arguments. This function will also be called in the class initialisation function, `myclass_make_new`, immediately after the argument checking. 

When implementing the operators, we have to keep a couple of things in mind. First, the `specialclass_myclass_type` has to be extended with the two methods, `.unary_op`, and `.binary_op`, where `.unary_op` is equal to the function that handles the unary operation (`specialclass_unary_op` in the example below), and `.binary_op` is equal to the function that deals with binary operations (`specialclass_binary_op` below). These two functions have the signatures 

```c
STATIC mp_obj_t specialclass_unary_op(mp_unary_op_t op, mp_obj_t self_in)
```
and

```c
STATIC mp_obj_t specialclass_binary_op(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs)
```
respectively, and we have to inspect the value of `op` in the implementation. This is done in the two `switch` statements. 

One more comment: if `.unary_op`, or `.binary_op` are defined for the class, then the handler function must have an implementation of all operators. This doesn't necessarily mean that you have to have all cases in the `switch`, but if you haven't, then there must be a default case with a reasonable return value, e.g., `MP_OBJ_NULL`, or `mp_const_none`, so as to indicate that that particular method is not available. 

```c
#include "py/runtime.h"
#include "py/obj.h"
#include "py/binary.h"

typedef struct _specialclass_myclass_obj_t {
	mp_obj_base_t base;
	int16_t a;
	int16_t b;
} specialclass_myclass_obj_t;

const mp_obj_type_t specialclass_myclass_type;

STATIC void myclass_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
	(void)kind;
	specialclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	mp_print_str(print, "myclass(");
	printf("%d, ", self->a);
	printf("%d)", self->b);
}

mp_obj_t create_new_myclass(uint16_t a, uint16_t b) {
	specialclass_myclass_obj_t *out = m_new_obj(specialclass_myclass_obj_t);
	out->base.type = &specialclass_myclass_type;
	out->a = a;
	out->b = b;
	return MP_OBJ_FROM_PTR(out);
}

STATIC mp_obj_t myclass_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
	mp_arg_check_num(n_args, n_kw, 2, 2, true);
	return create_new_myclass(mp_obj_get_int(args[0]), mp_obj_get_int(args[1]));
}

STATIC const mp_rom_map_elem_t myclass_locals_dict_table[] = {
};

STATIC MP_DEFINE_CONST_DICT(myclass_locals_dict, myclass_locals_dict_table);

STATIC mp_obj_t specialclass_unary_op(mp_unary_op_t op, mp_obj_t self_in) {
	specialclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
    switch (op) {
        case MP_UNARY_OP_BOOL: return mp_obj_new_bool((self->a > 0) && (self->b > 0));
        case MP_UNARY_OP_LEN: return mp_obj_new_int(2);
        default: return MP_OBJ_NULL; // op not supported
    }
}

STATIC mp_obj_t specialclass_binary_op(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs) {
    specialclass_myclass_obj_t *left_hand_side = MP_OBJ_TO_PTR(lhs);
	specialclass_myclass_obj_t *right_hand_side = MP_OBJ_TO_PTR(rhs);
	switch (op) {
		case MP_BINARY_OP_EQUAL:
			return mp_obj_new_bool((left_hand_side->a == right_hand_side->a) && (left_hand_side->b == right_hand_side->b));
		case MP_BINARY_OP_ADD:
			return create_new_myclass(left_hand_side->a + right_hand_side->a, left_hand_side->b + right_hand_side->b);
		case MP_BINARY_OP_MULTIPLY:
			return create_new_myclass(left_hand_side->a * right_hand_side->a, left_hand_side->b * right_hand_side->b);
		default:
			return MP_OBJ_NULL; // op not supported
	}
}

const mp_obj_type_t specialclass_myclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_specialclass,
	.print = myclass_print,
	.make_new = myclass_make_new,
	.unary_op = specialclass_unary_op, 
	.binary_op = specialclass_binary_op,
	.locals_dict = (mp_obj_dict_t*)&myclass_locals_dict,
};

STATIC const mp_map_elem_t specialclass_globals_table[] = {
	{ MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_specialclass) },
	{ MP_OBJ_NEW_QSTR(MP_QSTR_myclass), (mp_obj_t)&specialclass_myclass_type },	
};

STATIC MP_DEFINE_CONST_DICT (
	mp_module_specialclass_globals,
	specialclass_globals_table
);

const mp_obj_module_t specialclass_user_cmodule = {	
	.base = { &mp_type_module },
	.globals = (mp_obj_dict_t*)&mp_module_specialclass_globals,
};

MP_REGISTER_MODULE(MP_QSTR_specialclass, specialclass_user_cmodule, MODULE_SPECIALCLASS_ENABLED);

```

## Defining constants

Constants can be added to the locals dictionary as any other object. So, e.g., if we wanted to define the constant MAGIC, we could do that as follows

```c

#define MAGIC 42

STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_MAGIC), MP_ROM_INT(MAGIC) },
};
```
The constant would then be accessible in the interpreter as

```python

import test

test.MAGIC
```

## The complete code

Having seen how components of a class should be implemented, we can bring all parts together. We will define a class that has two methods, one with positional arguments, and one with keyword arguments, has a class-specific constant, and can deal with the `+` and `*` operators. You should be able to drop this code into your user-defined modules directory, and compile it without difficulties.

```c

```

# Creating new types

Sometimes it might be necessary to define your own data type. It is really nothing but a C structure with a couple of special fields. The steps required are very similar to those for classes. 
Take the following type definition, which could be regarded as the Cartesian components of a vector in three-dimensional space: 

```c
typedef struct _vector_obj_t {
	mp_obj_base_t base;
	float x, y, z;
} vector_obj_t;
```

Now, in order to see, how we can work with this structure, we are going to define a new type that simply stores the three values. The module will also have a method called `length`, returning the absolute value of the vector. Note that here we check the type of the argument, and bail out, if it is not a vector.

```c
#include <math.h>
#include "py/obj.h"
#include "py/runtime.h"

const mp_obj_type_t vector_type;

typedef struct _vector_obj_t {
    mp_obj_base_t base;
    float x, y, z;
} vector_obj_t;

STATIC mp_obj_t vector_length(mp_obj_t o_in) {
	if(!mp_obj_is_type(o_in, &vector_type)) {
		mp_raise_TypeError("argument is not a vector");
	}
	vector_obj_t *vector = MP_OBJ_TO_PTR(o_in);
	return mp_obj_new_float(sqrt(vector->x*vector->x + vector->y*vector->y + vector->z*vector->z));
}

STATIC MP_DEFINE_CONST_FUN_OBJ_1(vector_length_obj, vector_length);


STATIC void vector_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
	(void)kind;
	vector_obj_t *self = MP_OBJ_TO_PTR(self_in);
	printf("vector(%f, %f, %f)\n", self->x, self->y, self->z);
}

STATIC mp_obj_t vector_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
	mp_arg_check_num(n_args, n_kw, 3, 3, true);
	
	vector_obj_t *vector = m_new_obj(vector_obj_t);
	vector->base.type = &vector_type;
	vector->x = mp_obj_get_float(args[0]);
	vector->y = mp_obj_get_float(args[1]);
	vector->z = mp_obj_get_float(args[2]);
	return MP_OBJ_FROM_PTR(vector);
}

const mp_obj_type_t vector_type = {
	{ &mp_type_type },
	.name = MP_QSTR_vector,
	.print = vector_print,
	.make_new = vector_make_new,
};

STATIC const mp_rom_map_elem_t vector_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_vector) },
    { MP_OBJ_NEW_QSTR(MP_QSTR_vector), (mp_obj_t)&vector_type },	
    { MP_ROM_QSTR(MP_QSTR_length), MP_ROM_PTR(&vector_length_obj) },
};
STATIC MP_DEFINE_CONST_DICT(vector_module_globals, vector_module_globals_table);

const mp_obj_module_t vector_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&vector_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_vector, vector_user_cmodule, MODULE_VECTOR_ENABLED);


```

# Dealing with iterables


In python, an iterable is basically an object that you can have in a `for` loop:

```python
for item in my_iterable:
    print(item)
```

Amongs others, lists, tuples, and ranges are iterables, as are strings. The key is that these object have a special internal method, and iterator, attached to them. This iterator is responsible for keeping track of the index during the iteration, and serving the objects in the iterable one by one to the `for` loop. When writing our own iterable, we will look under the hood, and see how this all works at the C level. For now, we are going to discuss only, how we can *consume* the content of an iterable in the C code. 

## Iterating over built-in types

In order to demonstrate the use of an iterator, we are going to write a function that sums the square of the values in an iterable. The python version of the function could be something like this: 

In [11]:
def sumsq(some_iterable):
    return sum([item**2 for item in some_iterable])

sumsq([1, 2, 3, 4])

30

In C, the key is in the snippet

```c
mp_obj_iter_buf_t iter_buf;
mp_obj_t item, iterable = mp_getiter(o_in, &iter_buf);
while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
    // do something with the item just retrieved
}
```
This is more or less the equivalent of the `for item in some_iterable` instruction. In C, the `mp_obj_t` object `o_in` is the argument of our python function, which is turned into an iterable by passing it into the `mp_getiter` function. This function also needs a buffer object that is of type `mp_obj_iter_buf_t`. The buffer type is defined in `obj.h` as 

```c
typedef struct _mp_obj_iter_buf_t {
    mp_obj_base_t base;
    mp_obj_t buf[3];
} mp_obj_iter_buf_t;

```
where `.buf[2]` holds the index value, and this is how `mp_iternext` keeps track of the position in the loop. 

Having retrieved `item`, the rest the code is trivial: you do whatever you want to do with the value, and return at the very end. 

Now, what happens, if you pass a non-iterable object to the function? For a while, nothing. Everything will work till the point `item = mp_iternext(iterable)`, where the interpreter will raise a `TypeError` exception. So, on the python console, you can either enclose your function in a 

```python
try:
    sumsq(some_iterable)
except TypeError:
    print('something went terribly wrong`)
```

construct, or you can inspect the type of the variable at the C level. Unfortunately, there does not seem to be a type identifier for iterables in general, so you have to check, whether the argument is a list, tuple, range, etc. This can be done by calling the `MP_OBJ_IS_TYPE` macro, and see which Boolean it returns, if you pass `&mp_type_tuple`, `&mp_type_list`, `&mp_type_range` etc. to it, as we discussed in the section Object representation. 

The complete code listing of `consumeiterable.c` follows below. If you ask me, this is a lot of code just to replace a single-line python function.

```c
#include "py/obj.h"
#include "py/runtime.h"


STATIC mp_obj_t iterables_sumsq(mp_obj_t o_in) {
	mp_float_t _sum = 0.0, itemf;
	mp_obj_iter_buf_t iter_buf;
	mp_obj_t item, iterable = mp_getiter(o_in, &iter_buf);
	while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
		itemf = mp_obj_get_float(item);
		_sum += itemf*itemf;
	}
	return mp_obj_new_float(_sum);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_1(iterables_sumsq_obj, iterables_sumsq);

STATIC const mp_rom_map_elem_t iterables_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_iterables) },
    { MP_ROM_QSTR(MP_QSTR_sumsq), MP_ROM_PTR(&iterables_sumsq_obj) },
};
STATIC MP_DEFINE_CONST_DICT(iterables_module_globals, iterables_module_globals_table);

const mp_obj_module_t iterables_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&iterables_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_iterables, iterables_user_cmodule, MODULE_ITERABLES_ENABLED);
```

## Returning iterables

Let us suppose that the result of some operation is an iterable, e.g., a tuple, or a list. How would we return such an object? How about a function that returns the powers of its argument? In python

In [9]:
def powerit(base, exponent):
    return [base**e for e in range(0, exponent+1)]

powerit(2, 10)

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

and in C, `returniterable.c`, 
```c
#include "py/obj.h"
#include "py/runtime.h"

STATIC mp_obj_t powers_iterable(mp_obj_t base, mp_obj_t exponent) {
	int e = mp_obj_get_int(exponent);
	mp_obj_t tuple[e+1];
	int b = mp_obj_get_int(base), ba = 1;
	for(int i=0; i <= e; i++) {
		tuple[i] = mp_obj_new_int(ba);
		ba *= b;
	}
	return mp_obj_new_tuple(e+1, tuple);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_2(powers_iterable_obj, powers_iterable);

STATIC const mp_rom_map_elem_t returniterable_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_returniterable) },
    { MP_ROM_QSTR(MP_QSTR_powers), MP_ROM_PTR(&powers_iterable_obj) },
};
STATIC MP_DEFINE_CONST_DICT(returniterable_module_globals, returniterable_module_globals_table);

const mp_obj_module_t returniterable_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&returniterable_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_returniterable, returniterable_user_cmodule, MODULE_RETURNITERABLE_ENABLED);
```

As everything else, the elements of tuples and lists are objects of type `mp_obj_t`, so, after finding out how far we have got to go with the exponents, we declare an array of the required length.  Values are generated and assigned in the while loop. Since on the left hand side of the assignment we have an `mp_obj_t`, we convert the results with `mp_obj_new_int`. Once we are done with the computations, we return the array with `mp_obj_new_tuple`. This functions takes the array as the second argument, while the first argument specifies the length. 

If you happen to want to return a list instead of a tuple, all you have to do is use `mp_obj_new_list` instead at the very end. 

## Creating iterables

Having seen how we can consume the elements in an iterable, it is time to see what this `.getiter` magic is doing. So, let us create a new type, `square`, and make it iterable! `square` will be a binary array holding 16-bit integers, and the values will be 1, 4, 9, 16... We are interested only in the iterability of the object, and for this reason, we will implement only the `.getiter` special method, and skip `.binary_op`, and `.unary_op`. If needed, these can easily be added based on the discussion in Class special methods. 



In [24]:
%cd ../../micropython/ports/unix

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


In [28]:
!make USER_C_MODULES=../../../usermod CFLAGS_EXTRA=-DMODULE_MAKEITERABLE_ENABLED=1 all

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
GEN build/genhdr/moduledefs.h
GEN build/genhdr/qstr.i.last
GEN build/genhdr/qstr.split
GEN build/genhdr/qstrdefs.collected.h
QSTR not updated
GEN build/genhdr/qstrdefs.generated.h
CC ../../py/mpstate.c
CC ../../py/nlr.c
CC ../../py/nlrx86.c
CC ../../py/nlrx64.c
CC ../../py/nlrthumb.c
CC ../../py/nlrxtensa.c
CC ../../py/nlrsetjmp.c
CC ../../py/malloc.c
CC ../../py/gc.c
CC ../../py/pystack.c
CC ../../py/qstr.c
CC ../../py/vstr.c
CC ../../py/mpprint.c
CC ../../py/unicode.c
CC ../../py/mpz.c
CC ../../py/reader.c
CC ../../py/lexer.c
CC ../../py/parse.c
CC ../../py/scope.c
CC ../../py/compile.c
CC ../../py/emitcommon.c
CC ../../py/emitbc.c
CC ../../py/asmbase.c
CC ../../py/asmx64.c
CC ../../py/emitnx64.c
CC ../../py/asmx86.c
CC ../../py/emitnx86.c
CC ../../py/asmthumb.c
CC ../../py/emitnthumb.c
CC ../../py/emitinlinethumb.c
CC ../../py/asmarm.c
CC ../../py/emitnarm.c
CC ../../py/asmxtensa.c
CC ../../py/emitnxt

## Subscripts

We now know, how we construct something that can be passed to a `for` loop. This is a good start. But iterables have other very useful properties. For instance, have you ever wondered, what actually happens in the following snippet?

In [29]:
a = 'micropython'
a[5]

'p'

`a` is a string, therefore, an iterable. Where does the interpreter know from, that it has got to return `p`, when asked for `a[5]`? Or have you ever been curious to know, how the interpreter replaces `p` by `q`, if

In [31]:
a = [c for c in 'micropyton']
a[5] = 'q'
a

['m', 'i', 'c', 'r', 'o', 'q', 'y', 't', 'o', 'n']

is passed to it? If so, then it is your lucky day, because we are going to make our iterable class be able to deal with such request.

The code snippets above rely on a single special method, the subscription. In the C code of micropython, this method is called `subscr`, and it should be assigned in the class declaration, i.e., if we take `makeiterable.c` as our basis for the following discussion, then we would have to extend 

```c
const mp_obj_type_t iterable_array_type {
    ...
    .subscr = itarray_subscr
}
```
where the signature of `itarray_subscr` has the form 

```c
STATIC mp_obj_t itarray_subscr(mp_obj_t self_in, mp_obj_t index, mp_obj_t value)
```
If `.subscr` is not implemented, but you call 

```python
a[5]
```
all the same, then the interpreter is going to complain 

```pytb
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: 'itarray' object isn't subscriptable
```


# Profiling

There are times, when you might want to find out what resources (time and RAM) a particular operation requires. Not because you are nosy, but because the resources of a microcontroller are limited, therefore, if you are out of luck, the desired operation might not even fit within the constraints of the chip. In order to locate the bottleneck, you will need to do a bit of profiling. Or perhaps, a lot. This is what we are going to discuss now. 

Since you are not going to face serious difficulties when running micropython on a computer, profiling makes really sense only in the context of the microcontroller, so this might be a golden opportunity to brush up on how the firmware has to be compiled and uploaded. It is not by accident that we spent some time on this at the very beginning of this document. 


## Profiling in python

### Measuring time

If you are interested in the execution time of a complete function, you can measure it simply by making use of the python interpreter

```python

from utime import ticks_us, diff_ticks

now = ticks_us()
run_my_function()
then = diff_ticks(ticks_us(), now)

print("function run_my_function() took %d us to run"%then)

```

In fact, since our function is flanked by two other statements, this construct easily lends itself to a decorator implementation, as in (taken from ...)

```python
```


(If you need an even better estimate, you can get the ticks twice, and yank run_my_function() in the second pass: in this way, you would get the cost of measuring time itself:

```python

from utime import ticks_us, diff_ticks

now = ticks_us
then = diff_ticks(ticks_us(), now)

print("the time measurement took %d us"%then)

```
Then you subtract the results of the second measurement from those of the first.)


### The memory cost of a function

While time is money, RAM is gold. We shouldn't pass up on that! The `micropython` has a very handy function for 
printing a pretty summary of the state of the RAM. You would call it like 

```python
import micropython

micropyton.mem_info()
```
which will give you something like this

```python
mem: total=5748, current=1858, peak=3451
stack: 928 out of 80000
GC: total: 2072832, used: 2528, free: 2070304
 No. of 1-blocks: 61, 2-blocks: 4, max blk sz: 6, max free sz: 64686
```
If you call `mem_info()` after you executed your function, but before calling the garbage collector (if that is enabled, that is), then from the two reports, you can figure out how many bytes the function has eaten. 

## Profiling in C

With the profiling method above we can measure the cost of a complete function only, but we cannot say anything about individual instructions in the body. Execution time is definitely a significant issue, but even worse is the problem of RAM: it might happen that the function allocates a huge amount of memory, but cleans up properly before returning. Such a function would certainly wreak havoc, even if it is rather inocuous-looking from the outside. So, what do we do? We should probably just measure. It is not going to hurt. 

In the example below (`profiling.c`), I discuss both time and RAM measurements in a single module, because splitting them wouldn't be worth the trouble. The function whose behaviour we inspect does nothing, but calculate the length of a three-dimensional vector. With that, we can figure out, how much the assignment, and how much the actual calculation cost. 

```c
#include <math.h>
#include <stdio.h>
#include "py/obj.h"
#include "py/runtime.h"
#include "mphalport.h"  // needed for mp_hal_ticks_cpu()
#include "py/builtin.h" // needed for mp_micropython_mem_info()

STATIC mp_obj_t measure_cpu(mp_obj_t _x, mp_obj_t _y, mp_obj_t _z) {
	// first, measure the time taken up by the assignments
	mp_uint_t start = mp_hal_ticks_cpu();
	float x = mp_obj_get_float(_x);
	float y = mp_obj_get_float(_y);
	float z = mp_obj_get_float(_z);
	mp_uint_t stop = mp_hal_ticks_cpu();
	printf("time elapsed: %ld CPU cycles\n", stop - start);
	mp_micropython_mem_info(0, NULL);
	
	// measure the time used by the square root
	start = mp_hal_ticks_cpu();
	float hypo = sqrtf(x*x + y*y + z*z);
	stop = mp_hal_ticks_cpu();
	printf("time elapsed: %ld CPU cycles\n", stop - start);	
	mp_micropython_mem_info(0, NULL);
	return mp_obj_new_float(hypo);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_3(measure_cpu_obj, measure_cpu);

STATIC const mp_rom_map_elem_t profiling_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_profiling) },
    { MP_ROM_QSTR(MP_QSTR_measure), MP_ROM_PTR(&measure_cpu_obj) },
};
STATIC MP_DEFINE_CONST_DICT(profiling_module_globals, profiling_module_globals_table);

const mp_obj_module_t profiling_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&profiling_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_profiling, profiling_user_cmodule, MODULE_PROFILING_ENABLED);
```

The above-mentioned `mem_info()` function of the micropython module can directly be called from C: after including the `builtin.h` header, we can issue `mp_micropython_mem_info(0, NULL);`, defined in `modmicropython.c`, which will print everything we need. Although its signature contains two arguments, a `size_t` and an `mp_obj_t` pointer to the arguments, the function does not seem to care about them, so we can pass `0`, and `NULL` without any meaning. 

The function `mp_micropython_mem_info()` doesn't carry out any measurements in itself, it is only for pretty printing. The stats are collected by `mp_micropython_mem_total()`, `mp_micropython_mem_current()`, and `mp_micropython_mem_peak()`. Unfortunately, these functions are all declared STATIC, so we cannot call them from outsize `modmicropython.c`. If you need a numeric representation of the state of the RAM, you can make use of the `m_get_total_bytes_allocated(void)`, `m_get_current_bytes_allocated(void)`, and `m_get_peak_bytes_allocated(void)` functions of `py/malloc.c`. All three return a `size_t`. 

With the help of these three functions, we could, e.g., return the size of the consumed memory to the micropython interpreter at the end of our calculations. `measure_cpu()` could be modified as 

```c
STATIC mp_obj_t measure_cpu(mp_obj_t _x, mp_obj_t _y, mp_obj_t _z) {
    size_t start, middle, end;
    start = m_get_current_bytes_allocated();

    float x = mp_obj_get_float(_x);
    float y = mp_obj_get_float(_y);
    float z = mp_obj_get_float(_z);
    middle = m_get_current_bytes_allocated();

    float hypo = sqrtf(x*x + y*y + z*z);
    end = m_get_current_bytes_allocated();
    mp_obj_t tuple[4];
    tuple[0] = MP_OBJ_NEW_SMALL_INT(start);
    tuple[1] = MP_OBJ_NEW_SMALL_INT(middle);
    tuple[2] = MP_OBJ_NEW_SMALL_INT(end);
    tuple[3] = mp_obj_new_float(hypo);
    return mp_obj_new_tuple(4, tuple);
}
```

```bash
make BOARD=PYBV11 USER_C_MODULES=../../../usermod/ all
python ../../tools/pydfu.py -u build-PYBV11/firmware.dfu 
```

# Outline of a math library 

I indicated at the very beginning that my main motivation for writing this document was that I wanted to have a reasonable programming manual for the development of a math library. Since I couldn't find any manuals, I have turned the problem around, and written up, what I have learnt by developing the library. So, what should this library achieve, after all?

## Requirements

Recently, I have run into a limitations with the micropython interpreter. These difficulties were both speed, and 
RAM related. Therefore, I wanted to have something that can perform common mathematical calculations with in a pythonic way, with little burden on the RAM, and possibly fast. On PCs, such a library is called `numpy`, and it felt only natural to me to implement those aspects of `numpy` that would find an applications in the context of data acquisition of moderate volume: after all, no matter what, the microcontroller is not going to produce or collect huge amounts of data, but it might still be useful to process these data within the constraints of the microcontroller. Due to the nature of the data that would be dealt with, one can work with a very limited subset of `numpy`. 

Keeping these considerations in mind, I set my goals as follows:

* One should be able to vectorise standard mathematical functions, while these functions should still work for scalars, so 

```python
a = 1.0
sin(a)
```
and
```python
a = [1.0, 2.0, 3.0]
sin(a)
```
should both be valid expressions.

* There should be a binary container, (`ndarray`) for numbers that are results of vectorised operations, and one should be able to initialise a container by passing arbitrary iterables to a constructor (see ```sin([1, 2, 3])``` above).


* The array should be iterable, so that we can turn it into lists, tuples, etc.

* The relevant binary operations should work on arrays as in `numpy`, that is, e.g., 

```python
>>> a = ndarray([1, 2, 3, 4])
>>> (a + 1) + a*10
```
should evaluate to `ndarray([12, 23, 34, 45])`.

* 2D arrays (matrices) could be useful (see below), thus, the above-mentioned container should be able to store its `shape`.

* Having matrices, it is only natural to implement standard matrix operations (inversion, transposition etc.)

* These numerical arrays and matrices should have a reasonable visual representation (pretty printing)

* With the help of matrices, one can also think of polynomial fitting to measurement data

In [8]:
import IPython
js = "IPython.CodeCell.options_default.highlight_modes['magic_cpp'] = {'reg':[/^%%cpp/]};"
IPython.core.display.display_javascript(js, raw=True)

In [16]:
js = """
    (function () {
        var defaults = IPython.CodeCell.config_defaults || IPython.CodeCell.options_default;
        defaults.highlight_modes['magic_text/x-csrc'] = {'reg':[/^\\s*%%cpp/]};
    })();
    """
IPython.core.display.display_javascript(js, raw=True)

In [11]:
%%fortran

for i in range(123)

UsageError: Cell magic `%%fortran` not found.


In [17]:
%%cpp

void main() {
    uint16_t x = 15;
}

UsageError: Cell magic `%%cpp` not found.
